In [67]:
import pandas as pd
import os
import json
import glob
import sqlite3

In [27]:
with open('../parameters.json') as json_data:
    params = json.load(json_data)
    
DATA_DIR = params['DATA_DIR']
TMP_DIR = params['TMP_DIR']

In [70]:
with open('../parameters.json') as json_data:
    params = json.load(json_data)
    
    DATA_DIR = params['DIRECTORIES']['DATA_DIR']
    TMP_DIR = params['DIRECTORIES']['TMP_DIR']
    PRICE_FILE = params['FILES']['PRICE_FILE']
    STATION_FILE =params['FILES']['STATION_FILE']
    SERVICE_FILE =params['FILES']['SERVICE_FILE']
    GAS_DB = os.path.join(DATA_DIR, params['DATABASE']['GAS_DB'])

In [ ]:
#on va aller charger les données

In [ ]:
#c'est le repertoir ou je vais trouver mes data

In [2]:
DATA_DIR = '../../DATA'

In [3]:
TMP_DIR = '../../tmp'

In [5]:
df2014 = pd.read_csv(DATA_DIR + '/' + gas_files[0])

In [14]:
df2014 = pd.read_csv(os.path.join(DATA_DIR,gas_files[0]),
                    sep=';', header=None, dtype = {1: str}
                    )
df2014.columns = ['id_station', 'code_postal', 'type_station', 'latitude',
                  'longitude', 'date_releve', 'type_carburant',
                  'libelle_carburant', 'prix']

In [15]:
df2014.columns

Index(['id_station', 'code_postal', 'type_station', 'latitude', 'longitude',
       'date_releve', 'type_carburant', 'libelle_carburant', 'prix'],
      dtype='object')

In [16]:
df2014.dtypes

id_station             int64
code_postal           object
type_station          object
latitude             float64
longitude            float64
date_releve           object
type_carburant       float64
libelle_carburant     object
prix                 float64
dtype: object

In [17]:
df2014.head()

,id_station,code_postal,type_station,latitude,longitude,date_releve,type_carburant,libelle_carburant,prix
0,1000001,01000,R,4620114.0,519791.0,2014-01-02 11:08:03,1.0,Gazole,1304.0
1,1000001,01000,R,4620114.0,519791.0,2014-01-04 09:54:03,1.0,Gazole,1304.0
2,1000001,01000,R,4620114.0,519791.0,2014-01-05 10:27:09,1.0,Gazole,1304.0
3,1000001,01000,R,4620114.0,519791.0,2014-01-06 09:07:51,1.0,Gazole,1304.0
4,1000001,01000,R,4620114.0,519791.0,2014-01-07 09:23:56,1.0,Gazole,1304.0


In [43]:
import glob

gas_files = glob.glob(os.path.join(DATA_DIR, "Prix*.zip")) # on met l'étoile pour éviter de tout écrire, il prend tout

In [38]:
dfall = pd.DataFrame()

for year in gas_files:
    df = pd.read_csv(year,
                    sep=';', header=None, dtype = {1: str}
                    )
    df.columns = ['id_station', 'code_postal', 'type_station', 'latitude',
                  'longitude', 'date_releve', 'type_carburant',
                  'libelle_carburant', 'prix']
    dfall = pd.concat([dfall, df], axis=0)

assert dfall[1] == 10, "Pas le bon nombre de colonnes"


In [44]:
gas_files

['../../DATA\\Prix2014.zip',
 '../../DATA\\Prix2015.zip',
 '../../DATA\\Prix2016.zip']

In [31]:
dfall.shape

(9401381, 9)

In [39]:
stations_files = glob.glob(os.path.join(DATA_DIR, "Stations*.zip"))

In [40]:
stations_files

['../../DATA\\Stations2016.zip']

In [62]:
dfstations = pd.read_csv(stations_files[0],
                         sep='|',
                         dtype={1: str},
                         encoding='utf-8')
dfstations.columns = ['id_station', 'code_postal', 'type_station',
                     'latitude', 'longitude', 'adresse', 'ville']

In [54]:
dfstations.head()

,id_station,code_postal,type_station,latitude,longitude,adresse,ville
0,1000002,01000,R,4.621842e+06,522767.000000,16 Avenue de Marboz,BOURG-EN-BRESSE
1,1000004,01000,R,4.618836e+06,524447.000000,20 Avenue du Maréchal Juin,Bourg-en-Bresse
2,1000005,01000,R,4.620094e+06,519942.024022,642 Avenue de Trévoux,SAINT-DENIS-LèS-BOURG
3,1000006,01000,R,4.620754e+06,523758.000000,1 Boulevard John Kennedy,BOURG-EN-BRESSE
4,1000007,01000,R,4.620105e+06,524891.000000,360 AVENUE DU CAPITAINE DHONNE,Bourg-en-Bresse


In [59]:
dfservices = pd.read_csv(os.path.join(DATA_DIR, stations_file),
                         sep='|',
                         header=None,
                         dtype={1: str},
                         encoding='utf-8')
dfservices.columns = ['id_station', 'code_postal', 'type_station',
                     'latitude', 'longitude','services']

In [65]:
dfall.to_pickle(os.path.join(DATA_DIR, PRICE_FILE))
dfstations.to_pickle(os.path.join(DATA_DIR, STATION_FILE))
dfservices.to_pickle(os.path.join(DATA_DIR, SERVICE_FILE))

In [66]:
testdf = pd.read_pickle(os.path.join(DATA_DIR, PRICE_FILE))

In [ ]:
cnx = sqlite3('../../data/carburants.db')

In [71]:
cnx = sqlite3.connect(GAS_DB)

In [78]:
dfall.to_sql('prices', cnx, index=False, if_exists='replace')

In [79]:
dfservices.to_sql('services', cnx, index=False, if_exists='replace')

In [80]:
dfstations.to_sql('stations', cnx, index=False, if_exists='replace')

In [77]:
pd.read_sql_query('select * from prices where code_postal="13001"',
                 cnx)

,id_station,code_postal,type_station,latitude,longitude,date_releve,type_carburant,libelle_carburant,prix
0,13001001,13001,R,4330446.0,538957.0,2014-01-01 06:00:00,1.0,Gazole,1399.0
1,13001001,13001,R,4330446.0,538957.0,2014-01-07 06:00:00,1.0,Gazole,1389.0
2,13001001,13001,R,4330446.0,538957.0,2014-01-24 06:00:00,1.0,Gazole,1389.0
3,13001001,13001,R,4330446.0,538957.0,2014-01-25 06:00:00,1.0,Gazole,1384.0
4,13001001,13001,R,4330446.0,538957.0,2014-01-28 06:00:00,1.0,Gazole,1389.0
5,13001001,13001,R,4330446.0,538957.0,2014-01-29 06:00:00,1.0,Gazole,1389.0
6,13001001,13001,R,4330446.0,538957.0,2014-01-30 06:00:00,1.0,Gazole,1389.0
7,13001001,13001,R,4330446.0,538957.0,2014-02-14 06:00:00,1.0,Gazole,1394.0
8,13001001,13001,R,4330446.0,538957.0,2014-02-18 06:00:00,1.0,Gazole,1399.0
9,13001001,13001,R,4330446.0,538957.0,2014-02-26 06:00:00,1.0,Gazole,1394.0
